In [180]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

## IMPORTACIÓN

In [181]:
d_pm25 = pd.read_csv('Datos_Bogota/pm25/2014.csv', sep=';')
for i in range(2015,2019):
    d_temp = pd.read_csv('Datos_Bogota/pm25/' + str(i) + '.csv', sep=';', names=d_pm25.columns.values, skiprows= 1) 
    d_pm25 = d_pm25.append(d_temp)

In [182]:
d_2019 = pd.read_csv('Datos_Bogota/pm25/2019.csv', sep=';', usecols= range(12), names=d_pm25.columns.values, skiprows= 1)
d_pm25 = d_pm25.append(d_2019)


In [183]:
d_pm10 = pd.read_csv('Datos_Bogota/pm10/2014.csv', sep=';') 
for i in range(2015,2020):
    d_temp = pd.read_csv('Datos_Bogota/pm10/' + str(i) + '.csv', sep=';', names=d_pm10.columns.values, skiprows= 1) 
    d_pm10 = d_pm10.append(d_temp)

In [184]:
np.shape(d_pm25)

(72, 12)

In [185]:
def reset_index(datos):
    datos = datos.reset_index()
    datos = datos.iloc[:,1:]
    return datos

d_pm10 = reset_index(d_pm10)
d_pm25 = reset_index(d_pm25)

In [186]:
def to_col(datos, pm):
    columns = datos.columns
    
    abrv = []
    for c in columns[1:]:
        pos = 0 
        for i in range(len(c)):
            if c[i] == '-':
                pos = i+2
                break
        abrv.append(c[pos:])
    
    datos_new = datos[columns[:2]]
    datos_new['codigoSerial'] = abrv[0]
    datos_new = datos_new.rename(columns={columns[1]: pm})
    
    for i in range(2,len(columns)):
        datos_temp = datos[[columns[0],columns[i]]]
        datos_temp['codigoSerial'] = abrv[i-2]
        datos_temp = datos_temp.rename(columns={columns[i]: pm})
        datos_new = datos_new.append(datos_temp)
        
    return datos_new


datos = to_col(d_pm10, 'pm10')
datos2 = to_col(d_pm25, 'pm25')

/Users/PutosHackers/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/PutosHackers/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [187]:
np.shape(datos2)

(792, 3)

In [188]:
datos = datos.rename(columns={'Fecha':'Fecha_Hora'})
datos2 = datos2.rename(columns={'fecha':'Fecha_Hora'})

In [189]:
datos = datos.merge(datos2, on=['Fecha_Hora', 'codigoSerial'], how='left')

In [190]:
datos.head(5)

Fecha_Hora   pm10 codigoSerial  pm25
0    2014-01   95.0          Ken  36.0
1    2014-01   95.0          Ken   NaN
2    2014-02  103.0          Ken  41.0
3    2014-02  103.0          Ken  48.0
4    2014-03   94.0          Ken  42.0

## LIMPIAR

In [191]:
#conservar = ['Fecha_Hora', 'codigoSerial', 'pm25','pm10']
#datos = datos[conservar]

In [192]:
#pos_del = np.where(datos['Fecha'] == 'Fecha ')[0]
#datos = datos.drop(pos_del)

In [193]:
np.shape(datos)

(1008, 4)

In [194]:
datos['Fecha_Hora'] = pd.to_datetime(datos['Fecha_Hora'])

In [195]:
#datos = datos.loc[pd.DatetimeIndex(datos['Fecha_Hora']).year > 2018]

In [196]:
datos.iloc[:,1:-2] = datos.iloc[:,[1,-1]].astype(float)

In [197]:
#datos['pm25'].loc[ (datos['pm25']<= 0) | (datos['pm25']> 200)] = None

#datos['pm10'].loc[ (datos['pm10']<= 0) | (datos['pm10'] > 200)] = None

## DATOS GRANDES ERRONEOS

In [165]:
#datos['Semana'] = datos['Fecha_Hora'].dt.dayofweek

In [166]:
datos[['pm10', 'pm25']].mean()

pm10    45.327645
pm25    20.686177
dtype: float64

## CALENDARIO

In [210]:
130000*20*0.97*(10**-6)

2.522

In [198]:
import calendar
days = list(calendar.day_abbr)
months = list(calendar.month_abbr)[1:]

## SEMANAL

In [199]:
max_total = get_top_estaciones('Total',5)
max_total

(11, 4)


array(['Ken', 'LFer', 'MinA', 'SanC', 'Usaq'], dtype='<U4')

In [200]:
max_est_25

array(['Ken', 'MinA', 'SanC'], dtype='<U4')

In [201]:
#top n estaciones pm25

def get_top_estaciones(valor, n):
    
    #promedio total (mensual)
    d_total_mensual = datos.groupby(by= [datos['codigoSerial']]).mean()
    d_total_mensual = d_total_mensual.reset_index()
    d_total_mensual['Total'] = d_total_mensual['pm25'] + d_total_mensual['pm10']
    
    print(np.shape(d_total_mensual))
    
    #datos organizados
    total = d_total_mensual.sort_values(by= valor, ascending= False).head(n)
    
    #index del top
    max_est = np.array(list(set(total['codigoSerial'].values)))
    return np.sort(max_est)
    

max_est_25 = get_top_estaciones('pm25', 3)
#max_est_10 = get_top_estaciones('pm10', 3)
max_est_10 = max_est_25

(11, 4)


In [202]:
def get_semanal(datos):
    d_semanal = datos.groupby(by= [datos['codigoSerial'], datos['Semana']]).mean()
    d_semanal = d_semanal.reset_index()
    return d_semanal

#d_semanal = get_semanal(datos)

In [203]:
def get_estaciones(datos):
    return np.array(list(set(datos['codigoSerial'].values)))

In [204]:
colors = [
    'salmon',
    'mediumaquamarine',
    'slateblue',
    'grey',
    'lightpink',
    'khaki'
]

In [205]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def fig_bar(max_est_1, max_est_2, datos):
    
    d_semanal = get_semanal(datos)
    
    
    #figura
    fig = make_subplots(rows=2, 
                        cols=1, 
                        shared_xaxes=True, 
                        vertical_spacing = 0.03,
                        specs=[[{"secondary_y": True}],
                           [{"secondary_y": True}]]
                       )

    #opacidad
    op = 0.75


    #Traces 
    
    #top estaciones
    
    for i in range(np.size(max_est_1)):
        
        #query for data of each station
        datos_temp = d_semanal.loc[d_semanal['codigoSerial'] == max_est_1[i]]


        # Add traces

        fig.add_trace(go.Bar(x=datos_temp['Semana']+1, y=datos_temp['pm25'],
                             name='Estación ' + str(int(max_est_1[i])),
                             legendgroup = 'Estación ' + str(int(max_est_1[i])),
                             opacity= op,
                             marker_color = colors[i]
                            ),
                      row=1, col=1)



    for i in range(np.size(max_est_2)):
        
        showlegend = False
        color = colors[i]
        
        if np.isin(max_est_2[i], max_est_1) != True:
            showlegend = True
            color = colors[-1]
        
        #query for data of each station
        datos_temp = d_semanal.loc[d_semanal['codigoSerial'] == max_est_2[i]]

        fig.add_trace(go.Bar(x=datos_temp['Semana']+1, y=datos_temp['pm10'],
                             name='Estación ' + str(int(max_est_2[i])),
                             legendgroup = 'Estación ' + str(int(max_est_2[i])),
                             opacity= op,
                             marker_color = color,
                             showlegend=showlegend
                            ),
                     row=2, col=1)   
        
    #Total
    
    d_temp = datos.groupby(by = datos['Semana']).mean()[['pm25', 'pm10']]
    d_temp = d_temp.reset_index()

    fig.add_trace(go.Bar(x=d_temp['Semana']+1, y=d_temp['pm25'],
                         name='Promedio total ',
                         opacity= op,
                         legendgroup = 'Promedio total ',
                         marker_color = colors[i+1]
                                ),
                  row=1, col=1)    

    fig.add_trace(go.Bar(x=d_temp['Semana']+1, y=d_temp['pm10'],
                         name='Promedio total ',
                         legendgroup = 'Promedio total ',
                         opacity= op,
                         marker_color = colors[i+1],
                         showlegend=False),
                  row=2, col=1)   
    
    # Porcentaje diario
    
    d_temp[['pm25','pm10']] = d_temp[['pm25','pm10']]/np.sum(d_temp[['pm25','pm10']])
    
    fig.add_trace(go.Scatter(x=d_temp['Semana']+1, y=d_temp['pm25'],
                             name = 'Porcentaje dia de la semana',
                             legendgroup = 'Porcentaje dia de la semana',
                             mode = 'lines+markers',
                             showlegend = True,
                             line = dict(color='black', width=1.5, dash = 'dash')),
                  row=1, col=1, secondary_y=True,)
    
    fig.add_trace(go.Scatter(x=d_temp['Semana']+1, y=d_temp['pm10'],
                             name = 'Porcentaje dia de la semana',
                             legendgroup = 'Porcentaje dia de la semana',
                             mode = 'lines+markers',
                             showlegend = False,
                             line = dict(color='black', width=1.5, dash = 'dash')),
                  row=2, col=1, secondary_y=True,)
       
        

    
    #Figure layout
    
    fig.update_yaxes(title_text= 'pm 2.5',  row=1, col=1, secondary_y=False )
    fig.update_yaxes(title_text= 'pm 10', row=2, col=1, secondary_y=False)
    
    fig.update_yaxes(tickformat= '%', row=1, col=1, secondary_y=True)
    fig.update_yaxes(tickformat= '%', row=2, col=1, secondary_y=True)


    dim = 1200

    fig.update_layout(height=9*dim/16, width=dim, title= 'Promedio diario por estación (2019-2021)',
                      title_font_size=20, 
                      font_size=12,
                      legend=dict(
                                    orientation="h",
                                    yanchor="bottom",
                                    y=1.0,
                                    xanchor="right",
                                    x=0.8
                                )
                     )
    fig.show()

    #Write HTML file
    fig.write_html("Datos_2000_2020/comparativo_diario.html")

#d_temp = fig_bar(max_est_25, max_est_10, datos)

## MENSUAL

In [206]:
def get_mensual(datos):
    d_mensual = datos.groupby(by= [datos['codigoSerial'], pd.DatetimeIndex(datos['Fecha_Hora']).month]).mean()
    d_mensual = d_mensual.reset_index()
    return d_mensual

d_mensual = get_mensual(datos)

In [207]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def fig_bar(max_est_1, max_est_2, datos):
    
    d_semanal = get_mensual(datos)
    
    
    #figura
    fig = make_subplots(rows=2, 
                        cols=1, 
                        shared_xaxes=True, 
                        vertical_spacing = 0.03,
                        specs=[[{"secondary_y": True}],
                           [{"secondary_y": True}]]
                       )

    #opacidad
    op = 0.75


    #Traces 
    
    #top estaciones
    
    for i in range(np.size(max_est_1)):
        
        #query for data of each station
        datos_temp = d_semanal.loc[d_semanal['codigoSerial'] == max_est_1[i]]


        # Add traces

        fig.add_trace(go.Bar(x=datos_temp['Fecha_Hora'], y=datos_temp['pm25'],
                             name='Estación ' + max_est_1[i],
                             legendgroup = 'Estación ' + max_est_1[i],
                             opacity= op,
                             marker_color = colors[i]
                            ),
                      row=1, col=1)



    for i in range(np.size(max_est_2)):
        
        showlegend = False
        color = colors[i]
        
        if np.isin(max_est_2[i], max_est_1) != True:
            showlegend = True
            color = colors[-1]
        
        #query for data of each station
        datos_temp = d_semanal.loc[d_semanal['codigoSerial'] == max_est_2[i]]

        fig.add_trace(go.Bar(x=datos_temp['Fecha_Hora'], y=datos_temp['pm10'],
                             name='Estación ' + max_est_2[i],
                             legendgroup = 'Estación ' + max_est_2[i],
                             opacity= op,
                             marker_color = color,
                             showlegend=showlegend
                            ),
                     row=2, col=1)   
        
    #Total
    
    d_temp = d_mensual.groupby(by = d_mensual['Fecha_Hora']).mean()[['pm25', 'pm10']]
    d_temp = d_temp.reset_index()

    fig.add_trace(go.Bar(x=d_temp['Fecha_Hora'], y=d_temp['pm25'],
                         name='Promedio total ',
                         opacity= op,
                         legendgroup = 'Promedio total ',
                         marker_color = colors[i+1]
                                ),
                  row=1, col=1)    

    fig.add_trace(go.Bar(x=d_temp['Fecha_Hora'], y=d_temp['pm10'],
                         name='Promedio total ',
                         legendgroup = 'Promedio total ',
                         opacity= op,
                         marker_color = colors[i+1],
                         showlegend=False),
                  row=2, col=1)   
    
    # Porcentaje diario
    
    
    d_temp[['pm25','pm10']] = d_temp[['pm25','pm10']]/np.sum(d_temp[['pm25','pm10']])
    
    fig.add_trace(go.Scatter(x=d_temp['Fecha_Hora'], y=d_temp['pm25'],
                             name = 'Porcentaje mes del año',
                             legendgroup = 'Porcentaje mes del año',
                             mode = 'lines+markers',
                             showlegend = True,
                             line = dict(color='black', width=1.5, dash = 'dash')),
                  row=1, col=1, secondary_y=True,)
    
    fig.add_trace(go.Scatter(x=d_temp['Fecha_Hora'], y=d_temp['pm10'],
                             name = 'Porcentaje mes del año',
                             legendgroup = 'Porcentaje mes del año',
                             mode = 'lines+markers',
                             showlegend = False,
                             line = dict(color='black', width=1.5, dash = 'dash')),
                  row=2, col=1, secondary_y=True,)
    
    
    
    #Figure layout
    
    fig.update_yaxes(title_text= 'pm 2.5 (\u03BCg/m\u00b3)',  row=1, col=1, secondary_y=False )
    fig.update_yaxes(title_text= 'pm 10 (\u03BCg/m\u00b3)', row=2, col=1, secondary_y=False)
    
    fig.update_yaxes(tickformat= '%', row=1, col=1, secondary_y=True)
    fig.update_yaxes(tickformat= '%', row=2, col=1, secondary_y=True)
    
    fig.update_xaxes(tickmode = 'array',
                 tickvals = np.arange(1,13),
                 ticktext= months)


    dim = 1200

    fig.update_layout(height=9*dim/16, width=dim, title= 'Promedio mensual por estación (2014-2019)',
                      title_font_size=25, 
                      font_size=20,
                      legend=dict(
                                    orientation="h",
                                    yanchor="bottom",
                                    y=1.0,
                                    xanchor="right",
                                    x=0.95
                                )
                     )
    fig.show()

    #Write HTML file
    fig.write_html("Datos_Bogota/comparativo_mensual.html")

    
    return fig
    
d_temp = fig_bar(max_est_25, max_est_10, datos)

## CONTINUO

In [145]:
def get_datos_anual(datos):
    d_anual= datos
    
    #retrieve año mes y dia
    d_anual['Año'] =  pd.DatetimeIndex(d_anual['Fecha_Hora']).year
    d_anual['Mes'] =  pd.DatetimeIndex(d_anual['Fecha_Hora']).month
    
    #group by without adding index
    d_anual = d_anual.groupby(by= [d_anual['codigoSerial'], 
                   d_anual['Año'],
                   d_anual['Mes']
                  ],
             as_index = False).mean()
    
    #change type to str
    d_anual[['Año','Mes']] = d_anual[['Año','Mes']].astype(str)

    
    #add cero to one digit dates

    uno_len_m = np.where(d_anual['Mes'].str.len() == 1)
    d_anual['Mes'].iloc[uno_len_m] = '0' + d_anual['Mes'].iloc[uno_len_m]

    #add column in datetime format
    d_anual['Fecha_Hora'] = d_anual['Año'] + d_anual['Mes']
    
    #erase innecesary columns    
    #d_anual = d_anual.drop(['Semana','Mes'],axis =1)
    
    
    #column format to date
    d_anual['Fecha_Hora'] = pd.to_datetime(d_anual['Fecha_Hora'], format='%Y%m')
    
    return d_anual

d_anual = get_datos_anual(datos)

/Users/PutosHackers/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [178]:
def scatter_datos(max_est_1, max_est_2, datos):
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing = 0.01)

    op = 0.75

    s = 20

    w = 4

    span = 5
    #ADD TRACES 
    
    #TOP ESTACIONES

    ##1
    
    for i in range(np.size(max_est_1)):

        datos_temp = datos.loc[datos['codigoSerial'] == max_est_1[i]]


        # Add traces

        fig.add_trace(go.Scatter(x= datos_temp['Fecha_Hora'],y = datos_temp['pm25'],
                                 mode='markers',
                                 opacity= op,
                                 name = 'Estación ' + max_est_1[i],
                                 legendgroup = 'Estación ' + max_est_1[i],
                                 marker=dict(
                                     size=s,
                                     color = colors[i],)
                                ),
                      row=1, col=1)

    ##2
        
    for i in range(np.size(max_est_2)):
        
        showlegend = False
        color = colors[i]
        
        if np.isin(max_est_2[i], max_est_1) != True:
            showlegend = True
            color = colors[-1]
        
        #query for data of each station
        datos_temp = datos.loc[datos['codigoSerial'] == max_est_2[i]]

        fig.add_trace(go.Scatter(x= datos_temp['Fecha_Hora'], y = datos_temp['pm10'],
                                 mode='markers',
                                 opacity= op,
                                 name = 'Estación ' + max_est_2[i],
                                 legendgroup = 'Estación ' + max_est_2[i],
                                 showlegend = showlegend,
                                 marker=dict(
                                     size=s,
                                     color = color,)
                                ),
                      row=2, col=1)

    
    
    # MEDIA MÓVIL
    
    media = datos.groupby(by=datos['Fecha_Hora']).mean()
    media = media.reset_index()
    media['pm25']  = media['pm25'].ewm(span=span,adjust=False).mean()
    media['pm10']  = media['pm10'].ewm(span=span,adjust=False).mean()



    fig.add_trace(go.Scatter(x= media['Fecha_Hora'],y = media['pm25'],
                             mode='lines',
                             name = 'Media Móvil',
                             legendgroup = 'Media Móvil',
                             showlegend = True,
                             line = dict(color='black', width=w)),
                  row=1, col=1)



    fig.add_trace(go.Scatter(x= media['Fecha_Hora'],y = media['pm10'],
                             mode='lines',
                             name = 'Otras',
                             legendgroup = 'Media Móvil',
                             showlegend = False,
                             line = dict(color='black', width=w)),
                  row=2, col=1)

    
    #linea horizontal    
    
    
    val_1 = 10
    name_2 = 'Límite anual OMS'
    fig.add_trace(go.Scatter(x=[np.min(media['Fecha_Hora']), np.max(media['Fecha_Hora'])], y=[val_1,val_1],
                             name = name_2,
                             legendgroup = name_2,
                             mode = 'lines',
                             showlegend = True,
                             line = dict(color='red', width=w, dash = 'dash')),
                  row=1, col=1, secondary_y=False,)
    
    val_2 = 20
    
    fig.add_trace(go.Scatter(x=[np.min(media['Fecha_Hora']), np.max(media['Fecha_Hora'])], y=[val_2,val_2],
                             name = name_2,
                             legendgroup = name_2,
                             mode = 'lines',
                             showlegend = False,
                             line = dict(color='red', width=w, dash = 'dash')),
                  row=2, col=1, secondary_y=False,)
    
    
    # FIGURE LAYOUT
    
    
    dim = 1200
    
    fig.update_layout(height=9*dim/16, width=dim, title= 'Registros diarios por estación (2014-2019)',
                      title_font_size=25, 
                      font_size=20,
                      legend=dict(
                          orientation="h",
                          yanchor="bottom",
                          y=1.0,
                          xanchor="right", 
                          x = 0.95)                 
                 )
    
    fig.update_yaxes(title= 'pm 2.5 (\u03BCg/m\u00b3)',  row=1, col=1)
    fig.update_yaxes(title= 'pm 10 (\u03BCg/m\u00b3)', row=2, col=1)
    

    
    #fig.update_xaxes(range = ['2018-12-20','2021-02-15'])
    
    
    fig.show()
    
    
    #Write HTML file
    fig.write_html("Datos_Bogota/comparativo_estacion.html")
        
scatter_datos(max_est_25, max_est_10, d_anual)

# pm2.5 = 10 
#pm 10 = 

KeyError: 'codigoSerial'

## PORCENTUAL

In [177]:
def medias_moviles(max_est_1, max_est_2, datos):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing = 0.01,
                        specs=[[{"secondary_y": True}],
                           [{"secondary_y": True}]]
                   )
    
    op = 0.85

    s = 10

    w = 4
    
    span = 7
    
    # TRACES
    
    # MEDIA MÓVIL
    
    media = datos.groupby(by=datos['Fecha_Hora']).mean()
    media = media.reset_index()
    media['pm25']  = media['pm25'].ewm(span=span,adjust=False).mean()
    media['pm10']  = media['pm10'].ewm(span=span,adjust=False).mean()


    fig.add_trace(go.Scatter(x= media['Fecha_Hora'],y = media['pm25'],
                             mode='lines',
                             name = 'Total',
                             legendgroup = 'Total',
                             showlegend = True,
                             line = dict(color='black', width=w)),
                  row=1, col=1)



    fig.add_trace(go.Scatter(x= media['Fecha_Hora'],y = media['pm10'],
                             mode='lines',
                             name = 'Total',
                             legendgroup = 'Total',
                             showlegend = False,
                             line = dict(color='black', width=w)),
                  row=2, col=1)

    
    #TOP ESTACIONES

    ##1
    
    for i in range(np.size(max_est_1)):
    
        datos_temp = datos.loc[datos['codigoSerial'] == max_est_1[i]]

        datos_temp.loc[:,'pm25']  = datos_temp.loc[:,'pm25'].ewm(span=span,adjust=False).mean()
        datos_temp.loc[:,'pm10']  = datos_temp.loc[:,'pm10'].ewm(span=span,adjust=False).mean()

        # Add traces

        fig.add_trace(go.Scatter(x= datos_temp['Fecha_Hora'],y = datos_temp['pm25'],
                                 opacity= op,
                                 name = 'Estación ' + max_est_1[i],
                                 legendgroup = 'Estación ' + max_est_1[i],
                                 line = dict(color=colors[i], width=w)),
                      row=1, col=1)

    ##2
    
    for i in range(np.size(max_est_2)):
        
        showlegend = False
        color = colors[i]
        
        if np.isin(max_est_2[i], max_est_1) != True:
            showlegend = True
            color = colors[-1]
        

        datos_temp = datos.loc[datos['codigoSerial'] == max_est_2[i]]

        datos_temp.loc[:,'pm25']  = datos_temp.loc[:,'pm25'].ewm(span=span,adjust=False).mean()
        datos_temp.loc[:,'pm10']  = datos_temp.loc[:,'pm10'].ewm(span=span,adjust=False).mean()
        
        fig.add_trace(go.Scatter(x= datos_temp['Fecha_Hora'],y = datos_temp['pm10'],                     
                                 opacity= op,
                                 name = 'Estación ' + max_est_2[i],
                                 legendgroup = 'Estación ' + max_est_2[i],
                                 showlegend = showlegend,
                                 line = dict(color=color, width=w)),
                      row=2, col=1)

    
    
    #linea horizontal    
    
    val_1 = 10
    name_2 = 'Límite anual OMS'
    fig.add_trace(go.Scatter(x=[np.min(media['Fecha_Hora']), np.max(media['Fecha_Hora'])], y=[val_1,val_1],
                             name = name_2,
                             legendgroup = name_2,
                             mode = 'lines',
                             showlegend = True,
                             line = dict(color='red', width=w, dash = 'dash')),
                  row=1, col=1, secondary_y=False,)
    
    val_2 = 20
    
    fig.add_trace(go.Scatter(x=[np.min(media['Fecha_Hora']), np.max(media['Fecha_Hora'])], y=[val_2,val_2],
                             name = name_2,
                             legendgroup = name_2,
                             mode = 'lines',
                             showlegend = False,
                             line = dict(color='red', width=w, dash = 'dash')),
                  row=2, col=1, secondary_y=False,)
    
    
    
    # UPDATE LAYOUT
    fig.update_yaxes(title= 'pm 2.5 (\u03BCg/m\u00b3)',  row=1, col=1)
    fig.update_yaxes(title= 'pm 10 (\u03BCg/m\u00b3)', row=2, col=1)
    
    
    fig.update_yaxes(tickformat= '%', row=1, col=1, secondary_y=True)
    fig.update_yaxes(tickformat= '%', row=2, col=1, secondary_y=True)


    dim = 1200

    fig.update_layout(height=9*dim/16, width=dim, title= 'Media móvil por estación (2014-2019)',
                      title_font_size=25, 
                      font_size=20,
                      legend=dict(orientation="h",
                                  yanchor="bottom",
                                  y=1.0,
                                  xanchor="right",
                                  x=0.9
                                )
                     )
    
    fig.show()
    fig.write_html("Datos_Bogota/comparativo_media_móvil.html")

medias_moviles(max_est_25, max_est_10, d_anual)

KeyError: 'codigoSerial'

## DASHBOARD

In [39]:
def fig_bar(max_est_1, max_est_2, datos, col_num):
    
    d_semanal = get_mensual(datos)
    

    #opacidad
    op = 0.75


    #Traces 
    
    #top estaciones
    
    for i in range(np.size(max_est_1)):
        
        #query for data of each station
        datos_temp = d_semanal.loc[d_semanal['codigoSerial'] == max_est_1[i]]


        # Add traces

        fig.add_trace(go.Bar(x=datos_temp['Fecha_Hora'], y=datos_temp['pm25'],
                             name='Estación ' + max_est_1[i],
                             legendgroup = 'Estación ' + max_est_1[i],
                             opacity= op,
                             marker_color = colors[i]
                            ),
                      row=1, col=col_num)



    for i in range(np.size(max_est_2)):
        
        showlegend = False
        color = colors[i]
        
        if np.isin(max_est_2[i], max_est_1) != True:
            showlegend = True
            color = colors[-1]
        
        #query for data of each station
        datos_temp = d_semanal.loc[d_semanal['codigoSerial'] == max_est_2[i]]

        fig.add_trace(go.Bar(x=datos_temp['Fecha_Hora'], y=datos_temp['pm10'],
                             name='Estación ' + max_est_2[i],
                             legendgroup = 'Estación ' + max_est_2[i],
                             opacity= op,
                             marker_color = color,
                             showlegend=showlegend
                            ),
                     row=2, col=col_num)   
        
    #Total
    
    d_temp = d_mensual.groupby(by = d_mensual['Fecha_Hora']).mean()[['pm25', 'pm10']]
    d_temp = d_temp.reset_index()

    fig.add_trace(go.Bar(x=d_temp['Fecha_Hora'], y=d_temp['pm25'],
                         name='Promedio total ',
                         opacity= op,
                         legendgroup = 'Promedio total ',
                         marker_color = colors[i+1]
                                ),
                  row=1, col=col_num)    

    fig.add_trace(go.Bar(x=d_temp['Fecha_Hora'], y=d_temp['pm10'],
                         name='Promedio total ',
                         legendgroup = 'Promedio total ',
                         opacity= op,
                         marker_color = colors[i+1],
                         showlegend=False),
                  row=2, col=col_num)   
    
    # Porcentaje diario
    
    
    d_temp[['pm25','pm10']] = d_temp[['pm25','pm10']]/np.sum(d_temp[['pm25','pm10']])
    
    fig.add_trace(go.Scatter(x=d_temp['Fecha_Hora'], y=d_temp['pm25'],
                             name = 'Porcentaje mes del año',
                             legendgroup = 'Porcentaje mes del año',
                             mode = 'lines+markers',
                             showlegend = True,
                             line = dict(color='black', width=1.5, dash = 'dash')),
                  row=1, col=col_num, secondary_y=True,)
    
    fig.add_trace(go.Scatter(x=d_temp['Fecha_Hora'], y=d_temp['pm10'],
                             name = 'Porcentaje mes del año',
                             legendgroup = 'Porcentaje mes del año',
                             mode = 'lines+markers',
                             showlegend = False,
                             line = dict(color='black', width=1.5, dash = 'dash')),
                  row=2, col=col_num, secondary_y=True,)
    
    
def scatter_datos(max_est_1, max_est_2, datos, col_num):
    

    op = 0.75

    s = 20

    w = 4

    span = 5
    #ADD TRACES 
    
    #TOP ESTACIONES

    ##1
    
    for i in range(np.size(max_est_1)):

        datos_temp = datos.loc[datos['codigoSerial'] == max_est_1[i]]


        # Add traces

        fig.add_trace(go.Scatter(x= datos_temp['Fecha_Hora'],y = datos_temp['pm25'],
                                 mode='markers',
                                 opacity= op,
                                 name = 'Estación ' + max_est_1[i],
                                 legendgroup = 'Estación ' + max_est_1[i],
                                 marker=dict(
                                     size=s,
                                     color = colors[i],)
                                ),
                      row=1, col=col_num)

    ##2
        
    for i in range(np.size(max_est_2)):
        
        showlegend = False
        color = colors[i]
        
        if np.isin(max_est_2[i], max_est_1) != True:
            showlegend = True
            color = colors[-1]
        
        #query for data of each station
        datos_temp = datos.loc[datos['codigoSerial'] == max_est_2[i]]

        fig.add_trace(go.Scatter(x= datos_temp['Fecha_Hora'], y = datos_temp['pm10'],
                                 mode='markers',
                                 opacity= op,
                                 name = 'Estación ' + max_est_2[i],
                                 legendgroup = 'Estación ' + max_est_2[i],
                                 showlegend = showlegend,
                                 marker=dict(
                                     size=s,
                                     color = color,)
                                ),
                      row=2, col=col_num)

    
    
    # MEDIA MÓVIL
    
    media = datos.groupby(by=datos['Fecha_Hora']).mean()
    media = media.reset_index()
    media['pm25']  = media['pm25'].ewm(span=span,adjust=False).mean()
    media['pm10']  = media['pm10'].ewm(span=span,adjust=False).mean()



    fig.add_trace(go.Scatter(x= media['Fecha_Hora'],y = media['pm25'],
                             mode='lines',
                             name = 'Media Móvil',
                             legendgroup = 'Media Móvil',
                             showlegend = True,
                             line = dict(color='black', width=w)),
                  row=1, col=col_num)



    fig.add_trace(go.Scatter(x= media['Fecha_Hora'],y = media['pm10'],
                             mode='lines',
                             name = 'Otras',
                             legendgroup = 'Media Móvil',
                             showlegend = False,
                             line = dict(color='black', width=w)),
                  row=2, col=col_num)

    
    #linea horizontal    
    
    
    val_1 = 10
    name_2 = 'Límite anual OMS'
    fig.add_trace(go.Scatter(x=[np.min(media['Fecha_Hora']), np.max(media['Fecha_Hora'])], y=[val_1,val_1],
                             name = name_2,
                             legendgroup = name_2,
                             mode = 'lines',
                             showlegend = True,
                             line = dict(color='red', width=w, dash = 'dash')),
                  row=1, col=col_num, secondary_y=False,)
    
    val_2 = 20
    
    fig.add_trace(go.Scatter(x=[np.min(media['Fecha_Hora']), np.max(media['Fecha_Hora'])], y=[val_2,val_2],
                             name = name_2,
                             legendgroup = name_2,
                             mode = 'lines',
                             showlegend = False,
                             line = dict(color='red', width=w, dash = 'dash')),
                  row=2, col=col_num, secondary_y=False,)
    
    
    # FIGURE LAYOUT
    
    
    dim = 1200
    
    fig.update_layout(height=9*dim/16, width=dim, title= 'Registros diarios por estación (2014-2019)',
                      title_font_size=25, 
                      font_size=20,
                      legend=dict(
                          orientation="h",
                          yanchor="bottom",
                          y=1.0,
                          xanchor="right", 
                          x = 0.95)                 
                 )
    
    fig.update_yaxes(title= 'pm 2.5 (\u03BCg/m\u00b3)',  row=1, col=1)
    fig.update_yaxes(title= 'pm 10 (\u03BCg/m\u00b3)', row=2, col=1)
    

def medias_moviles(max_est_1, max_est_2, datos, col_num):
    
    op = 0.85

    s = 10

    w = 4
    
    span = 7
    
    # TRACES
    
    # MEDIA MÓVIL
    
    media = datos.groupby(by=datos['Fecha_Hora']).mean()
    media = media.reset_index()
    media['pm25']  = media['pm25'].ewm(span=span,adjust=False).mean()
    media['pm10']  = media['pm10'].ewm(span=span,adjust=False).mean()


    fig.add_trace(go.Scatter(x= media['Fecha_Hora'],y = media['pm25'],
                             mode='lines',
                             name = 'Total',
                             legendgroup = 'Total',
                             showlegend = True,
                             line = dict(color='black', width=w)),
                  row=1, col=col_num)



    fig.add_trace(go.Scatter(x= media['Fecha_Hora'],y = media['pm10'],
                             mode='lines',
                             name = 'Total',
                             legendgroup = 'Total',
                             showlegend = False,
                             line = dict(color='black', width=w)),
                  row=2, col=col_num)

    
    #TOP ESTACIONES

    ##1
    
    for i in range(np.size(max_est_1)):
    
        datos_temp = datos.loc[datos['codigoSerial'] == max_est_1[i]]

        datos_temp.loc[:,'pm25']  = datos_temp.loc[:,'pm25'].ewm(span=span,adjust=False).mean()
        datos_temp.loc[:,'pm10']  = datos_temp.loc[:,'pm10'].ewm(span=span,adjust=False).mean()

        # Add traces

        fig.add_trace(go.Scatter(x= datos_temp['Fecha_Hora'],y = datos_temp['pm25'],
                                 opacity= op,
                                 name = 'Estación ' + max_est_1[i],
                                 legendgroup = 'Estación ' + max_est_1[i],
                                 line = dict(color=colors[i], width=w)),
                      row=1, col=col_num)

    ##2
    
    for i in range(np.size(max_est_2)):
        
        showlegend = False
        color = colors[i]
        
        if np.isin(max_est_2[i], max_est_1) != True:
            showlegend = True
            color = colors[-1]
        

        datos_temp = datos.loc[datos['codigoSerial'] == max_est_2[i]]

        datos_temp.loc[:,'pm25']  = datos_temp.loc[:,'pm25'].ewm(span=span,adjust=False).mean()
        datos_temp.loc[:,'pm10']  = datos_temp.loc[:,'pm10'].ewm(span=span,adjust=False).mean()
        
        fig.add_trace(go.Scatter(x= datos_temp['Fecha_Hora'],y = datos_temp['pm10'],                     
                                 opacity= op,
                                 name = 'Estación ' + max_est_2[i],
                                 legendgroup = 'Estación ' + max_est_2[i],
                                 showlegend = showlegend,
                                 line = dict(color=color, width=w)),
                      row=2, col=col_num)

    
    
    #linea horizontal    
    
    val_1 = 10
    name_2 = 'Límite anual OMS'
    fig.add_trace(go.Scatter(x=[np.min(media['Fecha_Hora']), np.max(media['Fecha_Hora'])], y=[val_1,val_1],
                             name = name_2,
                             legendgroup = name_2,
                             mode = 'lines',
                             showlegend = True,
                             line = dict(color='red', width=w, dash = 'dash')),
                  row=1, col=col_num, secondary_y=False,)
    
    val_2 = 20
    
    fig.add_trace(go.Scatter(x=[np.min(media['Fecha_Hora']), np.max(media['Fecha_Hora'])], y=[val_2,val_2],
                             name = name_2,
                             legendgroup = name_2,
                             mode = 'lines',
                             showlegend = False,
                             line = dict(color='red', width=w, dash = 'dash')),
                  row=2, col=col_num, secondary_y=False,)
    
    

In [ ]:
    
    #Figure layout
    
    fig.update_yaxes(title_text= 'pm 2.5 (\u03BCg/m\u00b3)',  row=1, col=1, secondary_y=False )
    fig.update_yaxes(title_text= 'pm 10 (\u03BCg/m\u00b3)', row=2, col=1, secondary_y=False)
    
    fig.update_yaxes(tickformat= '%', row=1, col=1, secondary_y=True)
    fig.update_yaxes(tickformat= '%', row=2, col=1, secondary_y=True)
    
    fig.update_xaxes(tickmode = 'array',
                 tickvals = np.arange(1,13),
                 ticktext= months)


    dim = 1200

    fig.update_layout(height=9*dim/16, width=dim, title= 'Promedio mensual por estación (2014-2019)',
                      title_font_size=25, 
                      font_size=20,
                      legend=dict(
                                    orientation="h",
                                    yanchor="bottom",
                                    y=1.0,
                                    xanchor="right",
                                    x=0.95
                                )
                     )

In [76]:
help(make_subplots)

Help on function make_subplots in module plotly.subplots:

make_subplots(rows=1, cols=1, shared_xaxes=False, shared_yaxes=False, start_cell='top-left', print_grid=False, horizontal_spacing=None, vertical_spacing=None, subplot_titles=None, column_widths=None, row_heights=None, specs=None, insets=None, column_titles=None, row_titles=None, x_title=None, y_title=None, **kwargs)
    Return an instance of plotly.graph_objs.Figure with predefined subplots
    configured in 'layout'.
    
    Parameters
    ----------
    rows: int (default 1)
        Number of rows in the subplot grid. Must be greater than zero.
    
    cols: int (default 1)
        Number of columns in the subplot grid. Must be greater than zero.
    
    shared_xaxes: boolean or str (default False)
        Assign shared (linked) x-axes for 2D cartesian subplots
    
          - True or 'columns': Share axes among subplots in the same column
          - 'rows': Share axes among subplots in the same row
          - 'all': Sh

In [41]:
#figura
fig = make_subplots(rows=2, 
                    cols=3, 
                    shared_xaxes=True,
                    shared_yaxes=True,
                    vertical_spacing = 0.03,
                    specs=[[{"secondary_y": True}, {} , {}],
                       [{"secondary_y": True}, {}, {}]]
                   )

medias_moviles(max_est_25, max_est_10, d_anual, 3)

scatter_datos(max_est_25, max_est_10, d_anual, 2)

fig_bar(max_est_25, max_est_10, datos,1 )

fig.show()

## TO HTML

In [61]:


import chart_studio.tools as tls
tls.get_embed('https://plot.ly/~elizabethts/9/') #change to your url

#help(tls.get_embed)

PlotlyError: Because you didn't supply a 'file_id' in the call, we're assuming you're trying to snag a figure from a url. You supplied the url, 'https://plot.ly/~elizabethts/9/', we expected it to start with 'https://plotly.com'.
Run help on this function for more information.

In [54]:
username = 'davera' # your username
api_key = 'G66HMqzOff6JKS0t8GhD' # your api key - go to profile > settings > regenerate key
tls.set_credentials_file(username=username, api_key=api_key)

In [49]:
import chart_studio.plotly as py

In [50]:
py.plot(d_temp, filename= 'datosmensual', auto_open=False)

'https://plotly.com/~davera/2/'

In [45]:
type(d_temp)

NoneType